# ❄️❄️❄️ White Christmas? ❄️❄️❄️

When I was working at the Omaha World-Herald, my colleague Ben Vankat made [a cool page](http://dataomaha.com/media/news/2014/snow/) showing how often Omaha has a white Christmas (at least an inch of snow on the ground). The helpful folks at the local National Weather Service forecast office update [an HTML table](http://www.weather.gov/oax/christmas) each year with new data.

Not every NWS station does this, of course. But! It turns out that [NOAA has a climate data API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2), and I thought it'd be fun to write a Python script to collect historical Christmas snow data for Colorado Springs. And once you find the ID for your city, you can do the same thing.

### Request an access token from NOAA

Enter your email address [on this page](https://www.ncdc.noaa.gov/cdo-web/token) and you'll be sent an auth token.

Next, set this token as an environmental variable on your computer -- I called mine `NOAA_TOKEN`. (If you're not sure how to do this, [here's a good rundown](https://www.schrodinger.com/kb/1842).)

### Find your city's location ID

We're going to be asking for daily climate summary data from weather stations in Colorado Springs. NOAA has IDs for each location in its data. [According to the API documentation](https://www.ncdc.noaa.gov/cdo-web/webservices/v2#locations),
> Locations can be a specific latitude/longitude point such as a station, or a label representing a bounding area such as a city.

We're interested in cities, which can have multiple weather stations. A call to the endpoint `https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CITY` returns a paginated array of cities in the data. (I didn't see an option in the API docs to filter the results based on the name of the city.)

We _could_ loop through every page of results from that endpoint and programatically search each result set to find the station we need, but I'm lazy, so I just used a combination of `sortfield`, `offset` and `limit` parameters to trial-and-error my way to the page with "Colorado Springs" on it, then Ctrl+F'd to find Colorado Springs's ID (`CITY:US080003`). Here's what I ended up with.

In [1]:
import requests
from os import environ

TOKEN = environ.get('NOAA_TOKEN')

In [35]:
cities = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CITY&sortfield=name&offset=100&limit=500'

r = requests.get(cities, headers={'token': TOKEN})
print(r.text)

{"metadata":{"resultset":{"offset":100,"count":1980,"limit":500}},"results":[{"mindate":"1916-01-01","maxdate":"2017-12-13","name":"Athens, OH US","datacoverage":1,"id":"CITY:US390004"},{"mindate":"1903-12-01","maxdate":"2017-12-13","name":"Athens, TX US","datacoverage":1,"id":"CITY:US480004"},{"mindate":"1899-06-01","maxdate":"2017-12-13","name":"Atlanta, GA US","datacoverage":1,"id":"CITY:US130004"},{"mindate":"1881-01-01","maxdate":"2017-12-06","name":"Atyrau, KZ","datacoverage":0.9515,"id":"CITY:KZ000004"},{"mindate":"1906-04-01","maxdate":"2017-12-13","name":"Auburn, AL US","datacoverage":1,"id":"CITY:US010003"},{"mindate":"1994-08-01","maxdate":"2017-12-06","name":"Auckland, NZ","datacoverage":1,"id":"CITY:NZ000001"},{"mindate":"1891-07-01","maxdate":"2017-12-13","name":"Augusta, GA US","datacoverage":1,"id":"CITY:US130005"},{"mindate":"1886-09-01","maxdate":"2017-12-13","name":"Augusta, ME US","datacoverage":1,"id":"CITY:US230001"},{"mindate":"1893-01-01","maxdate":"2017-12-13",

### Determine data coverage

Now that we have our city ID, let's see how far back the data goes. According to the API docs, we can get some basic information on our chosen location by hitting the endpoint `https://www.ncdc.noaa.gov/cdo-web/api/v2/locations/{id}`. Let's do that.

In [43]:
from pprint import pprint

CITY_ID = 'CITY:US080003'
location_summary = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/locations/{id}'.format(id=CITY_ID)

r = requests.get(location_summary, headers={'token': TOKEN})

CITY_NAME = r.json()['name'].replace(' US', '')

pprint(r.json())

{'datacoverage': 1,
 'id': 'CITY:US080003',
 'maxdate': '2017-12-13',
 'mindate': '1894-05-01',
 'name': 'Colorado Springs, CO US'}


So the oldest data is from May 1894. _Nice_.

We're interested in data on Christmas each year, so let's build a string template and set a new variable, `MIN_YEAR`, equal to 1894. Later, we'll iterate through the years and populate that template to construct an API call.

In [7]:
MIN_YEAR = 1893
XMAS = '{}-12-25'

### Find the ID of the `dataset` we're interested in

To see all datasets available from this API, we can hit the endpoint `https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets`.

In [25]:
r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets', headers={'token': TOKEN})
pprint(r.json())

{'metadata': {'resultset': {'count': 11, 'limit': 25, 'offset': 1}},
 'results': [{'datacoverage': 1,
              'id': 'GHCND',
              'maxdate': '2017-12-11',
              'mindate': '1763-01-01',
              'name': 'Daily Summaries',
              'uid': 'gov.noaa.ncdc:C00861'},
             {'datacoverage': 1,
              'id': 'GSOM',
              'maxdate': '2017-11-01',
              'mindate': '1763-01-01',
              'name': 'Global Summary of the Month',
              'uid': 'gov.noaa.ncdc:C00946'},
             {'datacoverage': 1,
              'id': 'GSOY',
              'maxdate': '2017-01-01',
              'mindate': '1763-01-01',
              'name': 'Global Summary of the Year',
              'uid': 'gov.noaa.ncdc:C00947'},
             {'datacoverage': 0.95,
              'id': 'NEXRAD2',
              'maxdate': '2017-12-13',
              'mindate': '1991-06-05',
              'name': 'Weather Radar (Level II)',
              'uid': 'gov.noaa.ncd

We want `GHCND`, the daily summary. Let's set that as a variable.

In [12]:
DATASET = 'GHCND'

### Build the endpoint template

For every year, we're going to hit the same endpoint -- the only difference will be the `startdate` and `enddate` parameters. Let's make it a template.

In [11]:
ENDPOINT = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/' \
           'data?datasetid={}&locationid={}&startdate={}' \
           '&enddate={}&units=standard'

To get the actual data, we're making a call to the `data` endpoint and passing in these paramters:

- `datasetid`
- `locationid`
- `startdate`
- `enddate`

... we're also specifying that we want the results in _standard_ instead of _metric_ units.

### Write a function to parse daily summary data

Now we'll write a function that takes two arguments -- the target date and the city ID -- and returns a dictionary of snow-related data. Later, we'll call this function inside a loop that will iterate over all the Christmases for which there are data.

Each time it's called, the function will try to fetch the daily summary data on the `target_date` for all weather stations within the `location` -- we'll set Colorado Springs as the default location. The result set for each API call will be a JSON array of objects; each object, in turn, is a measurement observed that day by a weather station in that city.

We're interested in observations of two [`datatypes`](https://www.ncdc.noaa.gov/cdo-web/webservices/v2#dataTypes): _SNOW_ (snowfall) and _SNWD_ (snow depth). (If you're interested, you can page through the huge catalog of available datatypes at this endpoint: `https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?limit=999`.)

From there, we want our function to:
- Filter the observations to target just snowfall observed that day
- Filter the observations to target just snow depth observed that day
- For each list of snow-related observations, find the observation with the max value

In other words, we want to find and return the _maximum_ snowfall and _maximum_ snow depth observed in the location on that day. We'll also keep track of which station made each observation.

In [44]:
def get_daily_summary(target_date, location=CITY_ID):
    """Fetch and parse daily climate data for a location ID on a given date.
       Returns a dict w/ data on snowfall and snow depth.
    """
    
    # fill in the template for the endpoint
    ep = ENDPOINT.format(DATASET, location, target_date, target_date)
    
    # make the request
    r = requests.get(ep, headers={'token': TOKEN})
    
    try:
        # the key for the results is 'results'
        results = r.json()['results']

        # filter the results for snowfall
        snowfall = [x for x in results if x['datatype'] == 'SNOW']

        # filter the results for snow depth
        snow_depth = [x for x in results if x['datatype'] == 'SNWD']

        # not every date has these data types
        if snowfall:
            # find the max and set the vars
            max_snowfall = max(snowfall, key=lambda x: x['value'])
            station_sf = max_snowfall['station']
            max_sf = max_snowfall['value']
        else:
            # if no data, return '-' for each var
            station_sf = '-'
            max_sf = '-'

        if snow_depth:
            # find the max and set the vars
            max_snow_depth = max(snow_depth, key=lambda x: x['value'])
            station_sd = max_snow_depth['station']
            max_sd = max_snow_depth['value']
        else:
            # if no data, return '-' for each var
            station_sd = '-'
            max_sd = '-'

    except KeyError:
        station_sf = '-'
        max_sf = '-'
        station_sd = '-'
        max_sd = '-'

    return {
        'date': target_date,
        'station_sf': station_sf,
        'max_snowfall': max_sf,
        'station_sd': station_sd,
        'max_snowdepth': max_sd
    }

### Collect the data

Now we'll iterate through every year, call the function and dump the data to a CSV.

In [45]:
import csv
from time import sleep

XMAS_SNOW_CSV = 'xmas-colo-springs.csv'

In [13]:
with open(XMAS_SNOW_CSV, 'w') as f:

    headers = ['date', 'max_snowfall', 'max_snowdepth',
               'station_sf', 'station_sd']

    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()

    for i in range(MIN_YEAR, 2017):
        print('Grabbing data for Xmas', i)
        snow_data = get_daily_summary(XMAS.format(i))
        writer.writerow(snow_data)
        sleep(3)

Grabbing data for Xmas 1893
Grabbing data for Xmas 1894
Grabbing data for Xmas 1895
Grabbing data for Xmas 1896
Grabbing data for Xmas 1897
Grabbing data for Xmas 1898
Grabbing data for Xmas 1899
Grabbing data for Xmas 1900
Grabbing data for Xmas 1901
Grabbing data for Xmas 1902
Grabbing data for Xmas 1903
Grabbing data for Xmas 1904
Grabbing data for Xmas 1905
Grabbing data for Xmas 1906
Grabbing data for Xmas 1907
Grabbing data for Xmas 1908
Grabbing data for Xmas 1909
Grabbing data for Xmas 1910
Grabbing data for Xmas 1911
Grabbing data for Xmas 1912
Grabbing data for Xmas 1913
Grabbing data for Xmas 1914
Grabbing data for Xmas 1915
Grabbing data for Xmas 1916
Grabbing data for Xmas 1917
Grabbing data for Xmas 1918
Grabbing data for Xmas 1919
Grabbing data for Xmas 1920
Grabbing data for Xmas 1921
Grabbing data for Xmas 1922
Grabbing data for Xmas 1923
Grabbing data for Xmas 1924
Grabbing data for Xmas 1925
Grabbing data for Xmas 1926
Grabbing data for Xmas 1927
Grabbing data for Xm

### Load the data into `pandas` for analysis

In [39]:
import pandas as pd
import numpy as np

In [40]:
df = pd.read_csv(XMAS_SNOW_CSV)
df.max_snowfall = df.max_snowfall.replace('-', np.nan).astype(float)
df.max_snowdepth = df.max_snowdepth.replace('-', np.nan).astype(float)

### Kill out years with no data

Some years, there were no snowfall or snow depth observations that day. Let's exclude those.

In [54]:
total_years = len(df)
no_nulls = df.dropna(thresh=4)
no_nulls_count = len(no_nulls)

print(no_nulls_count, 'of', total_years, 'years have data')

118 of 123 years have data


### How many white Christmases?

... defined as having an inch or more of snow on the ground.

In [49]:
white_xmas = no_nulls[no_nulls.max_snowdepth >= 1.0]
white_xmas_count = len(white_xmas)

print('In the {} years for which there are data, {}, has had at least {} white Christmases ({:.2f}% of the time), according to my definition.'.format(
    no_nulls_count, CITY_NAME, white_xmas_count, (white_xmas_count / no_nulls_count) * 100
))

In the 118 years for which there are data, Colorado Springs, CO, has had at least 51 white Christmases (43.22% of the time), according to my definition.


### How many white Christmases (as defined by Pueblo NWS)?

The folks at the Pueblo NWS station [publish some Christmas-related climate data](https://www.weather.gov/pub/climateCosChristmasFacts), too, but they define a white Christmas as ">0.5 inches of snow falling on Christmas day with a least 1 inch on the ground." (Some of my results differ from theirs, FWIW.)

In [52]:
white_xmas_pub = no_nulls[(no_nulls.max_snowdepth >= 1.0) & (no_nulls.max_snowfall > 0.5)]
white_xmas_pub_count = len(white_xmas_pub)

print('In the {} years for which there are data, {}, has had at least {} white Christmases ({:.2f}% of the time), according to the definition of the Pueblo NWS office.'.format(
    no_nulls_count, CITY_NAME, white_xmas_pub_count, (white_xmas_pub_count / no_nulls_count) * 100
))

In the 118 years for which there are data, Colorado Springs, CO, has had at least 8 white Christmases (6.78% of the time), according to the definition of the Pueblo NWS office.


### Most snowfall and snow depth

In [78]:
most_snowfall = df.loc[df.max_snowfall.idxmax()]

print('Christmas in Colorado Springs with the most snowfall ({:.1f}"): {}'.format(most_snowfall.max_snowfall, most_snowfall.date[:4]))

Christmas in Colorado Springs with the most snowfall (4.8"): 1939


In [79]:
most_snowdepth = df.loc[df.max_snowdepth.idxmax()]

print('Christmas in Colorado Springs with the highest snow depth ({:.1f}"): {}'.format(most_snowdepth.max_snowdepth, most_snowdepth.date[:4]))

Christmas in Colorado Springs with the highest snow depth (23.0"): 1992
